In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [3]:
df.shape

(891, 12)

In [4]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
df = df.loc[df.Embarked.notna(), ['Survived','Pclass', 'Sex', 'Embarked']]

In [10]:
df.shape

(889, 4)

In [11]:
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Embarked    0
dtype: int64

In [12]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [13]:
x = df.loc[: , ['Pclass']]
y = df.Survived

In [14]:
x.shape

(889, 1)

In [15]:
y.shape

(889,)

In [16]:
from sklearn.linear_model import LogisticRegression

In [18]:
logreg = LogisticRegression(solver = 'lbfgs')

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
cross_val_score(logreg, x, y, cv = 5, scoring = 'accuracy').mean()

0.6783406335301212

In [21]:
y.value_counts(normalize = True)

Survived
0    0.617548
1    0.382452
Name: proportion, dtype: float64

In [22]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [26]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse_output = False)

In [28]:
ohe.fit_transform(df[['Sex']])

array([[0., 1.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [0., 1.]])

In [29]:
ohe.categories_

[array(['female', 'male'], dtype=object)]

In [30]:
x = df.drop('Survived', axis = 'columns')

In [31]:
x.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [32]:
from sklearn.compose import make_column_transformer

In [34]:
column_trans = make_column_transformer(
    (OneHotEncoder(), ['Sex', 'Embarked']),
    remainder = 'passthrough')

In [35]:
column_trans.fit_transform(x)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [36]:
from sklearn.pipeline import make_pipeline

In [37]:
pipe = make_pipeline(column_trans, logreg)

In [38]:
cross_val_score(pipe, x, y, cv = 5, scoring = 'accuracy').mean()

0.7727924839713071

In [39]:
x_new = x.sample(5, random_state = 99)
x_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [40]:
pipe.fit(x, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Sex', 'Embarked'])])),
                ('logisticregression', LogisticRegression())])

In [41]:
pipe.predict(x_new)

array([1, 0, 1, 1, 0], dtype=int64)